# 🥉 Bronze Layer - Data Exploration & Validation

**NYC Taxi Data Engineering Project - Microsoft Fabric**

## ⚠️ CRITICAL: Schema Mismatch Solution

NYC TLC thay đổi schema giữa các tháng, thậm chí trong cùng một năm:
- `yellow_tripdata_2023-01.parquet`: VendorID = BIGINT
- `yellow_tripdata_2023-10.parquet`: VendorID = INT32

**Giải pháp:** Đọc TỪNG FILE riêng lẻ, normalize schema, rồi union.

## 1. Setup & Configuration

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd

# Configuration
BRONZE_PATH = "Files/bronze/taxi_trips"

# Display settings
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
pd.set_option('display.max_columns', None)

print("✅ Setup completed")
print(f"📁 Bronze path: {BRONZE_PATH}")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 3, Finished, Available, Finished)

✅ Setup completed
📁 Bronze path: Files/bronze/taxi_trips


## 2. List All Parquet Files

In [2]:
# Get list of all parquet files
print("📂 Scanning for parquet files...")

# Method: Use Spark to list files
df_file_list = spark.read.format("binaryFile").option("pathGlobFilter", "*.parquet").load(BRONZE_PATH)
file_paths = [row.path for row in df_file_list.select("path").collect()]

print(f"📊 Found {len(file_paths)} parquet files:\n")
for f in sorted(file_paths):
    print(f"  • {f.split('/')[-1]}")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 4, Finished, Available, Finished)

📂 Scanning for parquet files...
📊 Found 59 parquet files:

  • yellow_tripdata_2021-01.parquet
  • yellow_tripdata_2021-02.parquet
  • yellow_tripdata_2021-03.parquet
  • yellow_tripdata_2021-04.parquet
  • yellow_tripdata_2021-05.parquet
  • yellow_tripdata_2021-06.parquet
  • yellow_tripdata_2021-07.parquet
  • yellow_tripdata_2021-08.parquet
  • yellow_tripdata_2021-09.parquet
  • yellow_tripdata_2021-10.parquet
  • yellow_tripdata_2021-11.parquet
  • yellow_tripdata_2021-12.parquet
  • yellow_tripdata_2022-01.parquet
  • yellow_tripdata_2022-02.parquet
  • yellow_tripdata_2022-03.parquet
  • yellow_tripdata_2022-04.parquet
  • yellow_tripdata_2022-05.parquet
  • yellow_tripdata_2022-06.parquet
  • yellow_tripdata_2022-07.parquet
  • yellow_tripdata_2022-08.parquet
  • yellow_tripdata_2022-09.parquet
  • yellow_tripdata_2022-10.parquet
  • yellow_tripdata_2022-11.parquet
  • yellow_tripdata_2022-12.parquet
  • yellow_tripdata_2023-01.parquet
  • yellow_tripdata_2023-02.parquet
  • y

## 3. Define Schema Normalization Function

In [3]:
def read_and_normalize_single_file(file_path):
    """
    Read a single parquet file and normalize its schema.
    This function reads the file WITHOUT inferring schema conflicts.
    """
    file_name = file_path.split('/')[-1]
    
    try:
        # Read single file
        df_raw = spark.read.parquet(file_path)
        
        # Get column names (case-insensitive mapping)
        cols_lower = {c.lower(): c for c in df_raw.columns}
        
        # Helper to get actual column name
        def get_col(name):
            return cols_lower.get(name.lower(), name)
        
        # Find airport_fee column (could be Airport_fee)
        airport_fee_col = None
        for c in df_raw.columns:
            if c.lower() == "airport_fee":
                airport_fee_col = c
                break
        
        # Select and cast all columns to STRING first, then to target types
        # This avoids the Parquet vectorized reader type conflict
        df_normalized = df_raw.selectExpr(
            "CAST(VendorID AS LONG) AS VendorID",
            "CAST(tpep_pickup_datetime AS TIMESTAMP) AS tpep_pickup_datetime",
            "CAST(tpep_dropoff_datetime AS TIMESTAMP) AS tpep_dropoff_datetime",
            "CAST(passenger_count AS DOUBLE) AS passenger_count",
            "CAST(trip_distance AS DOUBLE) AS trip_distance",
            "CAST(RatecodeID AS DOUBLE) AS RatecodeID",
            "CAST(store_and_fwd_flag AS STRING) AS store_and_fwd_flag",
            "CAST(PULocationID AS LONG) AS PULocationID",
            "CAST(DOLocationID AS LONG) AS DOLocationID",
            "CAST(payment_type AS LONG) AS payment_type",
            "CAST(fare_amount AS DOUBLE) AS fare_amount",
            "CAST(extra AS DOUBLE) AS extra",
            "CAST(mta_tax AS DOUBLE) AS mta_tax",
            "CAST(tip_amount AS DOUBLE) AS tip_amount",
            "CAST(tolls_amount AS DOUBLE) AS tolls_amount",
            "CAST(improvement_surcharge AS DOUBLE) AS improvement_surcharge",
            "CAST(total_amount AS DOUBLE) AS total_amount",
            f"CAST(COALESCE(congestion_surcharge, 0.0) AS DOUBLE) AS congestion_surcharge",
            f"CAST(COALESCE({airport_fee_col if airport_fee_col else 'NULL'}, 0.0) AS DOUBLE) AS airport_fee"
        )
        
        return df_normalized, df_raw.count()
        
    except Exception as e:
        print(f"    ❌ Error reading {file_name}: {str(e)[:100]}")
        return None, 0

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 5, Finished, Available, Finished)

## 4. Load All Files One by One

In [4]:
# Load each file individually
print("🔄 Loading files one by one...")
print("=" * 70)

all_dataframes = []
file_stats = []
total_loaded = 0

for i, file_path in enumerate(sorted(file_paths)):
    file_name = file_path.split('/')[-1]
    
    df_normalized, record_count = read_and_normalize_single_file(file_path)
    
    if df_normalized is not None:
        all_dataframes.append(df_normalized)
        total_loaded += record_count
        
        # Extract year-month from filename
        # Format: yellow_tripdata_YYYY-MM.parquet
        try:
            year_month = file_name.replace("yellow_tripdata_", "").replace(".parquet", "")
        except:
            year_month = "unknown"
        
        file_stats.append({
            "file": file_name,
            "year_month": year_month,
            "records": record_count
        })
        
        print(f"  ✅ [{i+1:2}/{len(file_paths)}] {file_name}: {record_count:,} records")
    else:
        print(f"  ❌ [{i+1:2}/{len(file_paths)}] {file_name}: FAILED")

print("=" * 70)
print(f"📊 Successfully loaded {len(all_dataframes)} files")
print(f"📊 Total records (estimated): {total_loaded:,}")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 6, Finished, Available, Finished)

🔄 Loading files one by one...
  ✅ [ 1/59] yellow_tripdata_2021-01.parquet: 1,369,769 records
  ✅ [ 2/59] yellow_tripdata_2021-02.parquet: 1,371,709 records
  ✅ [ 3/59] yellow_tripdata_2021-03.parquet: 1,925,152 records
  ✅ [ 4/59] yellow_tripdata_2021-04.parquet: 2,171,187 records
  ✅ [ 5/59] yellow_tripdata_2021-05.parquet: 2,507,109 records
  ✅ [ 6/59] yellow_tripdata_2021-06.parquet: 2,834,264 records
  ✅ [ 7/59] yellow_tripdata_2021-07.parquet: 2,821,746 records
  ✅ [ 8/59] yellow_tripdata_2021-08.parquet: 2,788,757 records
  ✅ [ 9/59] yellow_tripdata_2021-09.parquet: 2,963,793 records
  ✅ [10/59] yellow_tripdata_2021-10.parquet: 3,463,504 records
  ✅ [11/59] yellow_tripdata_2021-11.parquet: 3,472,949 records
  ✅ [12/59] yellow_tripdata_2021-12.parquet: 3,214,369 records
  ✅ [13/59] yellow_tripdata_2022-01.parquet: 2,463,931 records
  ✅ [14/59] yellow_tripdata_2022-02.parquet: 2,979,431 records
  ✅ [15/59] yellow_tripdata_2022-03.parquet: 3,627,882 records
  ✅ [16/59] yellow_tripda

## 5. Union All DataFrames

In [5]:
if len(all_dataframes) == 0:
    raise Exception("❌ No files were loaded successfully!")

print("🔄 Merging all DataFrames...")

# Union all - since all have same schema now, this should work
df_taxi = all_dataframes[0]
for df in all_dataframes[1:]:
    df_taxi = df_taxi.unionAll(df)

print("✅ Union completed")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 7, Finished, Available, Finished)

🔄 Merging all DataFrames...
✅ Union completed


In [6]:
# Cache and count
print("🔄 Caching and counting...")
df_taxi.cache()

total_records = df_taxi.count()
print(f"\n✅ Total Taxi Records: {total_records:,}")
print(f"📋 Columns: {len(df_taxi.columns)}")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 8, Finished, Available, Finished)

🔄 Caching and counting...

✅ Total Taxi Records: 194,457,948
📋 Columns: 19


## 6. Verify Schema

In [7]:
print("🔍 Final Unified Schema:")
df_taxi.printSchema()

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 9, Finished, Available, Finished)

🔍 Final Unified Schema:
root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = false)
 |-- airport_fee: double (nullable = false)



In [8]:
# Show sample
print("📋 Sample Data:")
display(df_taxi.limit(10))

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 10, Finished, Available, Finished)

📋 Sample Data:


SynapseWidget(Synapse.DataFrame, 2c995d38-416b-4fef-9517-754e8610dc8c)

## 7. Data Distribution by Year

In [9]:
print("📅 Records by Year:")

df_taxi.withColumn("year", F.year("tpep_pickup_datetime")) \
    .groupBy("year") \
    .agg(
        F.count("*").alias("record_count"),
        F.min("tpep_pickup_datetime").alias("min_date"),
        F.max("tpep_pickup_datetime").alias("max_date")
    ) \
    .orderBy("year") \
    .show()

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 11, Finished, Available, Finished)

📅 Records by Year:
+----+------------+-------------------+-------------------+
|year|record_count|           min_date|           max_date|
+----+------------+-------------------+-------------------+
|2001|          12|2001-01-01 00:03:14|2001-08-23 05:34:45|
|2002|         464|2002-10-21 00:13:16|2002-12-31 23:41:25|
|2003|          25|2003-01-01 00:02:39|2003-01-29 08:34:30|
|2004|           1|2004-04-04 04:08:30|2004-04-04 04:08:30|
|2007|           1|2007-12-05 18:45:00|2007-12-05 18:45:00|
|2008|         154|2008-12-31 00:00:00|2008-12-31 23:58:56|
|2009|         282|2009-01-01 00:00:11|2009-01-02 01:34:14|
|2011|           4|2011-01-31 23:33:10|2011-02-01 00:39:52|
|2012|           1|2012-02-22 16:42:09|2012-02-22 16:42:09|
|2014|           1|2014-11-19 08:18:40|2014-11-19 08:18:40|
|2020|          16|2020-12-31 13:44:02|2020-12-31 23:59:40|
|2021|    30903958|2021-01-01 00:00:04|2021-12-31 23:59:51|
|2022|    39655622|2022-01-01 00:00:08|2022-12-31 23:59:59|
|2023|    38310138|20

In [10]:
# Records by Year-Month
print("📅 Records by Year-Month:")

df_taxi.withColumn("year_month", F.date_format("tpep_pickup_datetime", "yyyy-MM")) \
    .groupBy("year_month") \
    .agg(F.count("*").alias("trips")) \
    .orderBy("year_month") \
    .show(50)

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 12, Finished, Available, Finished)

📅 Records by Year-Month:
+----------+-------+
|year_month|  trips|
+----------+-------+
|   2001-01|     11|
|   2001-08|      1|
|   2002-10|    439|
|   2002-12|     25|
|   2003-01|     25|
|   2004-04|      1|
|   2007-12|      1|
|   2008-12|    154|
|   2009-01|    282|
|   2011-01|      2|
|   2011-02|      2|
|   2012-02|      1|
|   2014-11|      1|
|   2020-12|     16|
|   2021-01|1369753|
|   2021-02|1371689|
|   2021-03|1925130|
|   2021-04|2171215|
|   2021-05|2507075|
|   2021-06|2834204|
|   2021-07|2822113|
|   2021-08|2788300|
|   2021-09|2963778|
|   2021-10|3463485|
|   2021-11|3472897|
|   2021-12|3214319|
|   2022-01|2463936|
|   2022-02|2979429|
|   2022-03|3627891|
|   2022-04|3599902|
|   2022-05|3588292|
|   2022-06|3557692|
|   2022-07|3174363|
|   2022-08|3152694|
|   2022-09|3183784|
|   2022-10|3675384|
|   2022-11|3252670|
|   2022-12|3399585|
|   2023-01|3066731|
|   2023-02|2914003|
|   2023-03|3403660|
|   2023-04|3288249|
|   2023-05|3513664|
|   2023-

## 8. Basic Statistics

In [11]:
print("📈 Basic Statistics:")

df_taxi.select(
    F.count("*").alias("total_records"),
    F.countDistinct("VendorID").alias("vendors"),
    F.min("tpep_pickup_datetime").alias("earliest"),
    F.max("tpep_pickup_datetime").alias("latest"),
    F.round(F.avg("trip_distance"), 2).alias("avg_distance"),
    F.round(F.avg("fare_amount"), 2).alias("avg_fare"),
    F.round(F.avg("tip_amount"), 2).alias("avg_tip"),
    F.round(F.avg("total_amount"), 2).alias("avg_total"),
    F.round(F.sum("total_amount"), 2).alias("total_revenue")
).show(truncate=False)

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 13, Finished, Available, Finished)

📈 Basic Statistics:
+-------------+-------+-------------------+-------------------+------------+--------+-------+---------+---------------+
|total_records|vendors|earliest           |latest             |avg_distance|avg_fare|avg_tip|avg_total|total_revenue  |
+-------------+-------+-------------------+-------------------+------------+--------+-------+---------+---------------+
|194457948    |5      |2001-01-01 00:03:14|2098-09-11 02:23:31|5.74        |16.31   |3.89   |25.11    |4.88249358712E9|
+-------------+-------+-------------------+-------------------+------------+--------+-------+---------+---------------+



## 9. Data Quality Checks

In [12]:
print("⚠️ Data Quality Issues:")

df_taxi.select(
    F.sum(F.when(F.col("trip_distance") < 0, 1).otherwise(0)).alias("negative_distance"),
    F.sum(F.when(F.col("fare_amount") < 0, 1).otherwise(0)).alias("negative_fare"),
    F.sum(F.when(F.col("total_amount") < 0, 1).otherwise(0)).alias("negative_total"),
    F.sum(F.when(F.col("trip_distance") == 0, 1).otherwise(0)).alias("zero_distance"),
    F.sum(F.when(F.col("trip_distance") > 500, 1).otherwise(0)).alias("distance_over_500"),
    F.sum(F.when(F.col("fare_amount") > 1000, 1).otherwise(0)).alias("fare_over_1000"),
    F.sum(F.when(F.col("PULocationID") < 1, 1).otherwise(0)).alias("invalid_pickup"),
    F.sum(F.when(F.col("DOLocationID") < 1, 1).otherwise(0)).alias("invalid_dropoff")
).show(truncate=False)

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 14, Finished, Available, Finished)

⚠️ Data Quality Issues:
+-----------------+-------------+--------------+-------------+-----------------+--------------+--------------+---------------+
|negative_distance|negative_fare|negative_total|zero_distance|distance_over_500|fare_over_1000|invalid_pickup|invalid_dropoff|
+-----------------+-------------+--------------+-------------+-----------------+--------------+--------------+---------------+
|0                |4305345      |2306499       |3781935      |6525             |213           |0             |0              |
+-----------------+-------------+--------------+-------------+-----------------+--------------+--------------+---------------+



## 10. Load Reference Data

### 10.1 Taxi Zones

In [13]:
LAKEHOUSE_PATH = "Files/bronze"

# Load taxi zones
print("🗺️ Loading Taxi Zones...")
try:
    df_zones = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{LAKEHOUSE_PATH}/taxi_zones/taxi_zone_lookup.csv")
except:
    try:
        df_zones = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/taxi_zones/*.csv")
    except:
        df_zones = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/taxi_zones")

print(f"📊 Total Zones: {df_zones.count()}")

df_zones.groupBy("Borough").count().orderBy(F.desc("count")).show()

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 15, Finished, Available, Finished)

🗺️ Loading Taxi Zones...
📊 Total Zones: 265
+-------------+-----+
|      Borough|count|
+-------------+-----+
|       Queens|   69|
|    Manhattan|   69|
|     Brooklyn|   61|
|        Bronx|   43|
|Staten Island|   20|
|          EWR|    1|
|      Unknown|    1|
|          N/A|    1|
+-------------+-----+



### 10.2 Weather Data

In [15]:
print("🌤️ Loading Weather Data...")
try:
    df_weather = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{LAKEHOUSE_PATH}/weather/central_park_weather.csv")
except:
    try:
        df_weather = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/weather/*.csv")
    except:
        df_weather = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/weather")

print(f"📊 Total Weather Records: {df_weather.count()}")

# Show columns to see what's available
print(f"📋 Columns: {df_weather.columns}")

# Calculate avg_temp from tmax and tmin since tavg doesn't exist
df_weather.select(
    F.min("date").alias("start"),
    F.max("date").alias("end"),
    F.round((F.avg("tmax") + F.avg("tmin")) / 2, 1).alias("avg_temp")
).show()

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 17, Finished, Available, Finished)

🌤️ Loading Weather Data...
📊 Total Weather Records: 1155
📋 Columns: ['station', 'name', 'date', 'awnd', 'prcp', 'snow', 'snwd', 'tmax', 'tmin']
+----------+----------+--------+
|     start|       end|avg_temp|
+----------+----------+--------+
|2021-01-01|2024-02-29|    56.2|
+----------+----------+--------+



### 10.3 Holidays Data

In [18]:
print("🎄 Loading Holidays Data...")
try:
    df_holidays = spark.read.option("header", "true").option("inferSchema", "true") \
        .csv(f"{LAKEHOUSE_PATH}/holidays/us_holidays.csv")
except:
    try:
        df_holidays = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/holidays/*.csv")
    except:
        df_holidays = spark.read.option("header", "true").option("inferSchema", "true") \
            .csv(f"{LAKEHOUSE_PATH}/holidays")

print(f"📊 Total Holiday Records: {df_holidays.count()}")
display(df_holidays.limit(15))

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 20, Finished, Available, Finished)

🎄 Loading Holidays Data...
📊 Total Holiday Records: 75


SynapseWidget(Synapse.DataFrame, d24ef464-e804-4fcf-808f-57abbb23f647)

## 11. Summary Report

In [19]:
zones_count = df_zones.count()
weather_count = df_weather.count()
holidays_count = df_holidays.count()

print("=" * 80)
print("📋 BRONZE LAYER DATA QUALITY SUMMARY REPORT")
print("=" * 80)

print(f"""
┌────────────────────────────────────────────────────────────────────────────┐
│ DATASET              │ RECORDS         │ COLUMNS │ FORMAT  │ STATUS       │
├────────────────────────────────────────────────────────────────────────────┤
│ NYC Taxi Trips       │ {total_records:>15,} │ {len(df_taxi.columns):>7} │ Parquet │ ✅ Ready     │
│ Taxi Zones           │ {zones_count:>15,} │ {len(df_zones.columns):>7} │ CSV     │ ✅ Ready     │
│ Weather Data         │ {weather_count:>15,} │ {len(df_weather.columns):>7} │ CSV     │ ✅ Ready     │
│ US Holidays          │ {holidays_count:>15,} │ {len(df_holidays.columns):>7} │ CSV     │ ✅ Ready     │
└────────────────────────────────────────────────────────────────────────────┘
""")

# Date range
date_stats = df_taxi.select(
    F.min("tpep_pickup_datetime").alias("min_dt"),
    F.max("tpep_pickup_datetime").alias("max_dt")
).collect()[0]

print(f"📅 Taxi Data Range: {str(date_stats['min_dt'])[:10]} to {str(date_stats['max_dt'])[:10]}")
print(f"📁 Files Loaded: {len(all_dataframes)}")

print("\n" + "=" * 80)
print("✅ Bronze Layer Exploration Complete!")
print("=" * 80)
print("\n📌 Next Step: Run 02_Silver_Layer_Transformation notebook")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 21, Finished, Available, Finished)

📋 BRONZE LAYER DATA QUALITY SUMMARY REPORT

┌────────────────────────────────────────────────────────────────────────────┐
│ DATASET              │ RECORDS         │ COLUMNS │ FORMAT  │ STATUS       │
├────────────────────────────────────────────────────────────────────────────┤
│ NYC Taxi Trips       │     194,457,948 │      19 │ Parquet │ ✅ Ready     │
│ Taxi Zones           │             265 │       4 │ CSV     │ ✅ Ready     │
│ Weather Data         │           1,155 │       9 │ CSV     │ ✅ Ready     │
│ US Holidays          │              75 │       3 │ CSV     │ ✅ Ready     │
└────────────────────────────────────────────────────────────────────────────┘

📅 Taxi Data Range: 2001-01-01 to 2098-09-11
📁 Files Loaded: 59

✅ Bronze Layer Exploration Complete!

📌 Next Step: Run 02_Silver_Layer_Transformation notebook


## 12. Cleanup

In [20]:
df_taxi.unpersist()
print("✅ Cleanup completed")
print("\n🎉 Notebook finished successfully!")

StatementMeta(, 202eae77-fa32-480a-aa63-ce39a607e3fd, 22, Finished, Available, Finished)

✅ Cleanup completed

🎉 Notebook finished successfully!
